# E-learning Recommendation System

This notebook demonstrates a recommendation system for personalized e-learning content using deep learning-based collaborative filtering.
We cover:
- Dataset preparation
- User-Item matrix construction
- Neural network model for collaborative filtering
- Training and evaluation
- Example recommendations

In [ ]:
!pip install pandas numpy torch scikit-learn matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Sample dataset
data = {
    'user_id': [1,1,2,2,3,3,4,4,5,5],
    'item_id': [101,102,101,103,102,104,101,104,103,105],
    'rating': [5,4,4,5,3,4,2,5,3,4]
}
df = pd.DataFrame(data)
df.head()

In [ ]:
num_users = df['user_id'].nunique()
num_items = df['item_id'].nunique()

user_mapping = {id:i for i,id in enumerate(df['user_id'].unique())}
item_mapping = {id:i for i,id in enumerate(df['item_id'].unique())}

df['user_idx'] = df['user_id'].map(user_mapping)
df['item_idx'] = df['item_id'].map(item_mapping)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
class RatingsDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['user_idx'].values)
        self.items = torch.tensor(df['item_idx'].values)
        self.ratings = torch.tensor(df['rating'].values, dtype=torch.float32)
    def __len__(self):
        return len(self.ratings)
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

train_dataset = RatingsDataset(train_df)
test_dataset = RatingsDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2)

In [ ]:
class CFModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_size=8):
        super(CFModel, self).__init__()
        self.user_emb = nn.Embedding(num_users, embedding_size)
        self.item_emb = nn.Embedding(num_items, embedding_size)
        self.fc = nn.Linear(embedding_size, 1)
    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        x = u * i
        return self.fc(x).squeeze()

model = CFModel(num_users, num_items)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Training
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for user, item, rating in train_loader:
        optimizer.zero_grad()
        pred = model(user, item)
        loss = criterion(pred, rating)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

In [ ]:
# Evaluation
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for user, item, rating in test_loader:
        pred = model(user, item)
        preds.extend(pred.tolist())
        actuals.extend(rating.tolist())

mse = np.mean((np.array(preds) - np.array(actuals))**2)
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Generate Recommendations for a user
user_id = 1
user_idx = user_mapping[user_id]
item_indices = torch.tensor([i for i in range(num_items)])
user_tensor = torch.tensor([user_idx]*num_items)

model.eval()
with torch.no_grad():
    scores = model(user_tensor, item_indices).numpy()

recommended_items = [k for k,v in sorted(zip(item_mapping.keys(), scores), key=lambda x: -x[1])]
print(f"Recommended items for user {user_id}: {recommended_items[:3]}")

# Conclusion
This notebook demonstrates a full pipeline for building a deep learning-based collaborative filtering recommendation system:
- User and item embeddings learned from data
- Predictions of user preferences
- Generating personalized recommendations
- Pipeline is ready for larger datasets and production